## Titanic Dataset For Kaggle Competition

In [1]:
import os
import sklearn
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from kaggle.api.kaggle_api_extended import KaggleApi

In [2]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

### Run The Below Two Cells To Download The Titanic Dataset From Kaggle (Requires API Key)

In [3]:
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [4]:
# api = KaggleApi()
# api.authenticate()

# api.competition_download_files("titanic", path=".", quiet=False)

# # Unzip it
# with zipfile.ZipFile("titanic.zip", "r") as zip_ref:
#     zip_ref.extractall("titanic_data")

# # Remove the zip file
# os.remove("titanic.zip")